# Preprocess IMU

## Modules

In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import dbpd
import os
import tsdf

## Constants

In [2]:
# tsdf files
subject = '0A0B82C94960D6DCABC1F597EC0BA657F4B0EDC320702BCEE3B6955CE924DE05'
sensor = 'IMU'
week_nr = '104'
segment_nr = '0001'

path_to_data = r'C:\Users\erik_\Documents\PhD\data\ppp\preprocessed\test_data'

input_path = os.path.join(path_to_data, '1.input', subject, sensor)
output_path = os.path.join(path_to_data, '2.filtered', subject, sensor)

base_filename = f'WatchData.{sensor}.Week{week_nr}.raw_segment{segment_nr}'
meta_filename = f'{base_filename}_meta.json'
values_filename = f'{base_filename}_samples.bin'
time_filename = f'{base_filename}_time.bin'
quality_filename = f'{base_filename}_quality.bin'

l_acceleration_cols = ['acceleration_x', 'acceleration_y', 'acceleration_z']

# filtering
fs = 100
lower_cutoff_frequency = 0.3
filter_order = 4

## Load data

In [6]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(input_path, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]

df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df.sample(2)

,time,acceleration_x,acceleration_y,acceleration_z,rotation_x,rotation_y,rotation_z
11633,10.0,1154,1392,-846,15,-31,-3
54130,10.0,-2489,331,421,69,800,-231


## Preprocess data

In [7]:
ppp = dbpd.PreprocessingPipeline(df, 'time', fs, fs, True)

ppp.df_sensors['time'] = ppp.transform_time_array(scale_factor=1000, do_convert_to_abs_time=True)

ppp.df_sensors = ppp.resample_data(metadata_samples.scale_factors)

for col in l_acceleration_cols:
    for result, side_pass in zip(['filt', 'grav'], ['lp', 'hp']):
        ppp.df_sensors[f'{result}_{col}'] = ppp.butterworth_filter(sensor_col=col, order=filter_order,
                                                                   cutoff_frequency=lower_cutoff_frequency,
                                                                   passband=side_pass)
        
    ppp.df_sensors = ppp.df_sensors.drop(columns=[col])
    ppp.df_sensors = ppp.df_sensors.rename(columns={f'filt_{col}': col})

# Store data

In [9]:
metadata_time.file_dir_path = output_path
metadata_samples.file_dir_path = output_path

In [10]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

tsdf.write_dataframe_to_binaries(output_path, ppp.df_sensors, [metadata_time, metadata_samples])
tsdf.write_metadata([metadata_time, metadata_samples], meta_filename)